In [1]:
!pip install google-generativeai pypdf requests transformers torch accelerate bitsandbytes psutil

In [2]:
import google.generativeai as genai
from pypdf import PdfReader
import requests
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os, gc, psutil
import bitsandbytes
from typing import Optional


tokenizer = None
model = None
RFP_text_files = []
RFP_response_text_files = []
LLM_verdict = [] # This remains global as results are appended to it

In [3]:
"""
import zipfile
import os

def unzip_files(dataset_folder):
    for filename in os.listdir(dataset_folder):
        if filename.endswith(".zip"):
            filepath = os.path.join(dataset_folder, filename)
            try:
                with zipfile.ZipFile(filepath, 'r') as zip_ref:
                    zip_ref.extractall(dataset_folder)
                print(f"Successfully unzipped {filename}")
            except zipfile.BadZipFile:
                print(f"Error: {filename} is not a valid zip file")
            except Exception as e:
                print(f"An error occurred while unzipping {filename}: {e}")

# Replace 'dataset' with the actual path to your dataset folder if it's different
dataset_folder_path = '/kaggle/input/dataset-tln/dataset'
unzip_files(dataset_folder_path)
"""

'\nimport zipfile\nimport os\n\ndef unzip_files(dataset_folder):\n    for filename in os.listdir(dataset_folder):\n        if filename.endswith(".zip"):\n            filepath = os.path.join(dataset_folder, filename)\n            try:\n                with zipfile.ZipFile(filepath, \'r\') as zip_ref:\n                    zip_ref.extractall(dataset_folder)\n                print(f"Successfully unzipped {filename}")\n            except zipfile.BadZipFile:\n                print(f"Error: {filename} is not a valid zip file")\n            except Exception as e:\n                print(f"An error occurred while unzipping {filename}: {e}")\n\n# Replace \'dataset\' with the actual path to your dataset folder if it\'s different\ndataset_folder_path = \'/kaggle/input/dataset-tln/dataset\'\nunzip_files(dataset_folder_path)\n'

In [4]:
RFP_PDF_files = []
RFP_response_PDF_files = []
for i in range(15,24):
  RFP_PDF_files.append(f'/kaggle/input/dataset-tln-2/dataset_2/{i}_(DCE and Answer)/{i}_CCTP.pdf')
  RFP_response_PDF_files.append(f'/kaggle/input/dataset-tln-2/dataset_2/{i}_(DCE and Answer)/{i}_Response.pdf')

In [5]:
RFP_text_files = []
RFP_response_text_files = []

In [6]:
def read_RFP_file(PDF_file):
  text = ""
  reader = PdfReader(PDF_file)
  for page in reader.pages:
     text+= page.extract_text()
  return text

for PDF_file in RFP_PDF_files :
  RFP_text_files.append(read_RFP_file(PDF_file))
for PDF_file in RFP_response_PDF_files :
  RFP_response_text_files.append(read_RFP_file(PDF_file))

In [7]:
import re
def clean_llama_output(text):
    """Clean up extra spaces and formatting issues from Llama output"""
    # Remove excessive spaces between words
    text = re.sub(r'\s+', ' ', text)
    
    # Fix French spacing issues around punctuation
    text = re.sub(r'\s+([,.;:!?])', r'\1', text)
    text = re.sub(r'([.!?])\s*([A-Z])', r'\1 \2', text)
    
    # Remove leading/trailing spaces
    text = text.strip()
    
    return text

In [8]:
def get_memory_usage():
    """Get current memory usage in MB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

def clear_gpu_cache():
    """Clear GPU cache if CUDA is available"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

def force_garbage_collection():
    """Force garbage collection to free unused memory"""
    gc.collect()
    clear_gpu_cache()

def check_memory_threshold(threshold_mb: float = 8000):
    """Check if memory usage exceeds threshold and force cleanup if needed"""
    current_memory = get_memory_usage()
    if current_memory > threshold_mb:
        print(f"Memory usage high ({current_memory:.1f}MB), forcing cleanup...")
        force_garbage_collection()
        new_memory = get_memory_usage()
        print(f"Memory after cleanup: {new_memory:.1f}MB")
        return True
    return False

In [9]:

def get_prompt_template(rfp_text, response_text) :
 return f"""

Ta tâche est d’évaluer si une réponse est adaptée à un RFP (appel d’offres). Tu dois donner deux choses :
1. Une **note sur 10**
2. Une **probabilité entre 0 et 1 que la réponse soit acceptée. 
Elle est basée sur la cohérence de la réponse et son adéquation à l'appel d'offre .**

Forme de sortie obligatoire:
Note: <nombre entre 0 et 10>
Probabilité: <nombre entre 0 et 1, basée sur la cohérence de la réponse>
Justification: <texte court>

RFP: {rfp_text}
Réponse: {response_text}

"""


In [10]:

def generate_ideas_batch(rfp_text: str, response_text: str) -> str:
    """Memory-optimized batch generation with automatic cleanup"""
    global tokenizer, model
    
    if tokenizer is None or model is None:
        return ""
    
    # Check memory before processing
    initial_memory = get_memory_usage()
    print(f"Memory at start: {initial_memory:.1f}MB")
    
    try:
        # Truncate inputs to fit context window
        rfp_tokens = tokenizer.encode(rfp_text, max_length=150, truncation=True)
        response_tokens = tokenizer.encode(response_text, max_length=150, truncation=True)
        
        # Decode back to truncated text
        rfp_text_short = tokenizer.decode(rfp_tokens, skip_special_tokens=True)
        response_text_short = tokenizer.decode(response_tokens, skip_special_tokens=True)
        
        # Clear intermediate variables
        del rfp_tokens, response_tokens
        
        # Construct prompt
        prompt = get_prompt_template(rfp_text_short, response_text_short)
        
        # Tokenize input with memory management
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=500)
        
        # Move to device only when needed
        if hasattr(model, 'device'):
            inputs = inputs.to(model.device)
        
        print(f"Input length: {inputs.input_ids.shape[1]} tokens")
        
        # Generate with memory-efficient settings
        with torch.no_grad():
            # Clear cache before generation
            clear_gpu_cache()
            
            outputs = model.generate(
                inputs.input_ids,
                max_new_tokens=200,
                do_sample=True,
                temperature=0.9,
                top_p=0.9,
                pad_token_id=tokenizer.eos_token_id,
                attention_mask=inputs.attention_mask,
                # Memory-efficient generation settings
                use_cache=True,  # Reuse key-value cache
                low_memory=True if hasattr(model, 'low_memory') else False
            )
        
        # Decode output
        full_output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_text = full_output_text[len(prompt):].strip()
        
        # Explicit cleanup of large tensors
        del inputs, outputs, full_output_text
        
        return generated_text
        
    except Exception as e:
        print(f"Error in generation: {e}")
        return ""
    
    finally:
        # Always cleanup after processing
        force_garbage_collection()
        final_memory = get_memory_usage()
        print(f"Memory after processing: {final_memory:.1f}MB")

In [11]:
def generate_ideas(i):
    """
    Processes a single RFP and response pair.
    This function acts as a wrapper to call the batch processing function
    for a single item.
    """
    global RFP_text_files, RFP_response_text_files, LLM_verdict

    # Extract the single RFP and response for this iteration
    single_rfp_text = RFP_text_files[i]
    single_response_text = RFP_response_text_files[i]

    # Call the batch processing function with a list containing only the current item
    # This maintains the batch processing logic even for a single item.
    results = generate_ideas_batch(single_rfp_text, single_response_text)
    
    # Append the result(s) to the global LLM_verdict list
    LLM_verdict.append(results) # Use extend as generate_ideas_batch returns a list

def main():
    """
    Main function for processing RFP files, adapted to the requested loop structure.
    """
    print("Starting main processing loop...")
    for i in range(len(RFP_text_files)):
        print(f"Processing RFP pair {i+1}/{len(RFP_text_files)}")
        generate_ideas(i)
    
    if LLM_verdict:
        print("\n--- Example output (first verdict) ---")
        print(LLM_verdict[0])
        print("\n--- All verdicts generated ---")
        # You might want to print all verdicts for review, or save them to a file
        # for verdict_idx, verdict_text in enumerate(LLM_verdict):
        #     print(f"Verdict {verdict_idx}: {verdict_text}\n---")
    else:
        print("No responses generated.")

In [12]:
"""!zip -r /content/dataset.zip /content/dataset
files.download("/content/dataset.zip")"""


'!zip -r /content/dataset.zip /content/dataset\nfiles.download("/content/dataset.zip")'

In [13]:
#!pip install -U bitsandbytes

In [14]:
if __name__ == "__main__":
    # --- Model and Tokenizer Loading ---
    model_id = "HuggingFaceH4/zephyr-7b-beta" # Recommended choice

    try:
        print(f"Attempting to load model: {model_id}")
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            device_map="auto", 
            trust_remote_code=False 
        )
        model.eval()
        
        print(f"Model '{model_id}' loaded successfully with quantization.")

    except Exception as e : 
        print(e)

    main()

Attempting to load model: HuggingFaceH4/zephyr-7b-beta


2025-07-23 11:01:19.984627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753268480.006323     272 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753268480.012921     272 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Model 'HuggingFaceH4/zephyr-7b-beta' loaded successfully with quantization.
Starting main processing loop...
Processing RFP pair 1/9
Memory at start: 1443.6MB
Input length: 483 tokens
Memory after processing: 1914.3MB
Processing RFP pair 2/9
Memory at start: 1914.3MB
Input length: 482 tokens
Memory after processing: 1916.1MB
Processing RFP pair 3/9
Memory at start: 1916.1MB
Input length: 483 tokens
Memory after processing: 1916.1MB
Processing RFP pair 4/9
Memory at start: 1916.1MB
Input length: 483 tokens
Memory after processing: 1916.1MB
Processing RFP pair 5/9
Memory at start: 1916.1MB
Input length: 482 tokens
Memory after processing: 1916.1MB
Processing RFP pair 6/9
Memory at start: 1916.1MB
Input length: 481 tokens
Memory after processing: 1937.8MB
Processing RFP pair 7/9
Memory at start: 1937.8MB
Input length: 482 tokens
Memory after processing: 1937.8MB
Processing RFP pair 8/9
Memory at start: 1937.8MB
Input length: 482 tokens
Memory after processing: 1946.4MB
Processing RFP pair

In [15]:
# Save all verdicts to a file
with open("llm_verdicts_1.txt", "w", encoding="utf-8") as f:
    for i, verdict in enumerate(LLM_verdict, 1):
        if verdict != '' : 
            f.write(f"--- Verdict {i} ---\n")
            f.write(verdict + "\n\n")

    
    print("All responses saved to llm_verdicts.txt.")


All responses saved to llm_verdicts.txt.


In [16]:
print(len(RFP_PDF_files))


9


In [17]:
print(LLM_verdict)

["Réponse: \nCette réponse est assez générale et ne fournit pas de détails précis sur la sécurité, la conformité et la continuité des services. Cependant, elle énumère ces critères de manière claire, ce qui est une bonne chose. Cela montre que la compagnie est au moins consciente des principales préoccupations de TV5MONDE et s'efforce de les adresser. En conséquence, j'accorderai une note de 7 sur 10 et une probabilité de 0,75 d'acceptation. Cependant, une réponse plus détaillée serait préférable pour maximiser la probabilité d'acceptation. Justification: La note est élevée car les principales préoccupations de TV5MONDE sont abordées, mais il manque", "La réponse est adaptée au RFP puisque l'on peut constater la bonne conformité avec la\ndemande en matière de profils d'expertise (cyber sécurité opérationnelle et test d'intrusion).\nCependant, il est à noter que le document manque de détail sur la façon dont ces expertises\nseront exercées, ainsi que sur les tâches exactes à accomplir. 